## 1. Introduction

This is an Exploratory Data Analysis (EDA) Kernel for Lyft Motion Prediction for Autonomous Vehicles competition dataset. 

![](https://self-driving.lyft.com/wp-content/uploads/2020/06/blog_3.jpg)


We start with the analysis preparation, which requires, for this competition, to install and load several packages for load and manage l5kit dataset.  
We follow with data exploration, reviewing the agents, the scenes, the frames and following with inspection of the animated scenes.

## 2. Analysis preparation

### 2.1. Install & load packages

We will have to install l5kit to access the data.

<img src="https://www.l5kit.org/_images/pipeline.png" width=800></img>


In [ ]:
!pip install --upgrade pip
!pip install pymap3d==2.1.0
!pip install -U l5kit

In [ ]:
import os
import numpy as np
import pandas as pd
from l5kit.data import ChunkedDataset, LocalDataManager
from l5kit.dataset import EgoDataset, AgentDataset
from l5kit.rasterization import build_rasterizer
from l5kit.configs import load_config_data
from l5kit.visualization import draw_trajectory, TARGET_POINTS_COLOR
from l5kit.geometry import transform_points
from l5kit.data import PERCEPTION_LABELS
from tqdm import tqdm
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns 
from matplotlib import animation, rc
from matplotlib.ticker import MultipleLocator
from IPython.display import display, clear_output
import PIL
from IPython.display import HTML

rc('animation', html='jshtml')

### 2.2. Configuration

We set the local dataset configurations before accessing it.
We set the path for the l5kit data folder by setting the environment variable `L5KIT_DATA_FOLDER` and we load the lyft configuration files from a `yaml` file from an external dataset. 

In [ ]:
os.environ["L5KIT_DATA_FOLDER"] = "/kaggle/input/lyft-motion-prediction-autonomous-vehicles"
cfg = load_config_data("/kaggle/input/lyft-config-files/visualisation_config.yaml")
print(cfg)

## 3. Load data

We load, using the l5kit local data manager, the dataset. L5kit uses [zarr](https://zarr.readthedocs.io/en/stable/) data format; here the arrays are divided into chunks and compressed.

In [ ]:
# local data manager
dm = LocalDataManager()
# set dataset path
dataset_path = dm.require(cfg["val_data_loader"]["key"])
# load the dataset; this is a zarr format, chunked dataset
chunked_dataset = ChunkedDataset(dataset_path)
# open the dataset
chunked_dataset.open()
print(chunked_dataset)

## 4. Data exploration


### 4.1. Explore the dataset

We load and show entities in the dataset. 

### 4.1.1. Agents

We start with the agents.

In [ ]:
agents = chunked_dataset.agents
agents_df = pd.DataFrame(agents)
agents_df.columns = ["data"]; features = ['centroid', 'extent', 'yaw', 'velocity', 'track_id', 'label_probabilities']

for i, feature in enumerate(features):
    agents_df[feature] = agents_df['data'].apply(lambda x: x[i])
agents_df.drop(columns=["data"],inplace=True)
print(f"agents dataset: {agents_df.shape}")
agents_df.head()

The fields in the agents dataset are the following:  

* centroid - the agent position (in plane - two dimmensions);   
* extent - the agent dimmensions (three dimmensions, let's called length, width, height);  
* yaw - the agent oscilation/twist about the vertical plane;   
* velocity - the speed of the agent - in euclidian space;   
* track_id - index of track associated to the agent;   
* label_probabilities - gives the probability for the agent to belong to one of 17 different agent type; we will explore these labels in a moment;    


Let's look to the distribution of few of these values. 

#### Centroid distribution

In [ ]:
agents_df['cx'] = agents_df['centroid'].apply(lambda x: x[0])
agents_df['cy'] = agents_df['centroid'].apply(lambda x: x[1])

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,8))
plt.scatter(agents_df['cx'], agents_df['cy'], marker='+')
plt.xlabel('x', fontsize=11); plt.ylabel('y', fontsize=11)
plt.title("Centroids distribution")
plt.show()

#### Extent distribution

In [ ]:
agents_df['ex'] = agents_df['extent'].apply(lambda x: x[0])
agents_df['ey'] = agents_df['extent'].apply(lambda x: x[1])
agents_df['ez'] = agents_df['extent'].apply(lambda x: x[2])

In [ ]:
sns.set_style('whitegrid')

fig, ax = plt.subplots(1,3,figsize=(16,5))
plt.subplot(1,3,1)
plt.scatter(agents_df['ex'], agents_df['ey'], marker='+')
plt.xlabel('ex', fontsize=11); plt.ylabel('ey', fontsize=11)
plt.title("Extent: ex-ey")
plt.subplot(1,3,2)
plt.scatter(agents_df['ey'], agents_df['ez'], marker='+', color="red")
plt.xlabel('ey', fontsize=11); plt.ylabel('ez', fontsize=11)
plt.title("Extent: ey-ez")
plt.subplot(1,3,3)
plt.scatter(agents_df['ez'], agents_df['ex'], marker='+', color="green")
plt.xlabel('ez', fontsize=11); plt.ylabel('ex', fontsize=11)
plt.title("Extent: ez-ex")
plt.show();

#### Yaw

Let's see yaw distribution.

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,8))
sns.distplot(agents_df['yaw'],color="magenta")
plt.title("Yaw distribution")
plt.show()

#### Velocity

Let's look to velocity distribution.

In [ ]:
agents_df['vx'] = agents_df['velocity'].apply(lambda x: x[0])
agents_df['vy'] = agents_df['velocity'].apply(lambda x: x[1])

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,8))
plt.title("Velocity distribution")
plt.scatter(agents_df['vx'], agents_df['vy'], marker='.', color="red")
plt.xlabel('vx', fontsize=11); plt.ylabel('vy', fontsize=11)
plt.show();

#### track id


In [ ]:
print("Number of tracks: ", agents_df.track_id.nunique())
print("Entries per track id (first 10): \n", agents_df.track_id.value_counts()[0:10])

Let's look to the distribution of the label probabilities.

In [ ]:
probabilities = agents["label_probabilities"]
labels_indexes = np.argmax(probabilities, axis=1)
counts = []
for idx_label, label in enumerate(PERCEPTION_LABELS):
    counts.append(np.sum(labels_indexes == idx_label))

agents_df = pd.DataFrame()
for count, label in zip(counts, PERCEPTION_LABELS):
    agents_df = agents_df.append(pd.DataFrame({'label':label, 'count':count},index=[0]))
agents_df = agents_df.reset_index().drop(columns=['index'], axis=1)

In [ ]:
print(f"agents probabilities dataset: {agents_df.shape}")
agents_df  

There are 4 different active agents present in the dataset, as following:  
* PERCEPTION_LABEL_UNKNOWN - majority;  
* PERCEPTION_LABEL_CAR;  
* PERCEPTION_LABEL_CYCLIST;  
* PERCEPTION_LABEL_PEDESTRIAN.


Let's look to their distribution:

In [ ]:
f, ax = plt.subplots(1,1, figsize=(10,4))
plt.scatter(agents_df['label'], agents_df['count']+1, marker='*')
plt.xticks(rotation=90, size=8)
plt.xlabel('Perception label')
plt.ylabel(f'Agents count')
plt.title("Agents perception label values count distribution")
plt.grid(True)
ax.set(yscale="log")
plt.show()

### 4.1.2. Scenes

Let's look now to the scenes.

In [ ]:
scenes = chunked_dataset.scenes
scenes_df = pd.DataFrame(scenes)
scenes_df.columns = ["data"]; features = ['frame_index_interval', 'host', 'start_time', 'end_time']
for i, feature in enumerate(features):
    scenes_df[feature] = scenes_df['data'].apply(lambda x: x[i])
scenes_df.drop(columns=["data"],inplace=True)
print(f"scenes dataset: {scenes_df.shape}")
scenes_df.head()

In [ ]:
f, ax = plt.subplots(1,1, figsize=(6,4))
sns.countplot(scenes_df.host)
plt.xlabel('Host')
plt.ylabel(f'Count')
plt.title("Scenes host count distribution")
plt.show()

Let's show the scenes frame index succesion, on the same graph with the host.

In [ ]:
scenes_df['frame_index_start'] = scenes_df['frame_index_interval'].apply(lambda x: x[0])
scenes_df['frame_index_end'] = scenes_df['frame_index_interval'].apply(lambda x: x[1])
scenes_df.head()

In [ ]:
f, ax = plt.subplots(1,1, figsize=(8,8))
spacing = 498
minorLocator = MultipleLocator(spacing)
ax.yaxis.set_minor_locator(minorLocator)
ax.xaxis.set_minor_locator(minorLocator)
plt.xlabel('Start frame index')
plt.ylabel(f'End frame index')
plt.grid(which = 'minor')
plt.title("Frames scenes start and end index (grouped per host)")
sns.scatterplot(scenes_df['frame_index_start'], scenes_df['frame_index_end'], marker='|',  hue=scenes_df['host'])
plt.show()

### 4.1.3. Frames

We are now looking to the frames.


In [ ]:
frames_df = pd.DataFrame(chunked_dataset.frames)
frames_df.columns = ["data"]; features = ['timestamp', 'agent_index_interval', 'traffic_light_faces_index_interval', 
                                          'ego_translation','ego_rotation']
for i, feature in enumerate(features):
    frames_df[feature] = frames_df['data'].apply(lambda x: x[i])
frames_df.drop(columns=["data"],inplace=True)
print(f"frames dataset: {frames_df.shape}")
frames_df.head()

The frames are described by:  
* timestamp;  
* agent index interval;   
* traffic light faces index interval;   
* ego translation;   
* ego rotation;   


Let's look to ego translations.

#### Ego translations

In [ ]:
frames_df['dx'] = frames_df['ego_translation'].apply(lambda x: x[0])
frames_df['dy'] = frames_df['ego_translation'].apply(lambda x: x[1])
frames_df['dz'] = frames_df['ego_translation'].apply(lambda x: x[2])

In [ ]:
sns.set_style('whitegrid')
plt.figure()

fig, ax = plt.subplots(1,3,figsize=(16,5))

plt.subplot(1,3,1)
plt.scatter(frames_df['dx'], frames_df['dy'], marker='+')
plt.xlabel('dx', fontsize=11); plt.ylabel('dy', fontsize=11)
plt.title("Translations: dx-dy")
plt.subplot(1,3,2)
plt.scatter(frames_df['dy'], frames_df['dz'], marker='+', color="red")
plt.xlabel('dy', fontsize=11); plt.ylabel('dz', fontsize=11)
plt.title("Translations: dy-dz")
plt.subplot(1,3,3)
plt.scatter(frames_df['dz'], frames_df['dx'], marker='+', color="green")
plt.xlabel('dz', fontsize=11); plt.ylabel('dx', fontsize=11)
plt.title("Translations: dz-dx")

fig.suptitle("Ego translations in 2D planes of the 3 components (dx,dy,dz)", size=14)
plt.show();

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(16,5))
colors = ['magenta', 'orange', 'darkblue']; labels= ["dx", "dy", "dz"]
for i in range(0,3):
    df = frames_df['ego_translation'].apply(lambda x: x[i])
    plt.subplot(1,3,i + 1)
    sns.distplot(df, hist=False, color = colors[ i ])
    plt.xlabel(labels[i])
fig.suptitle("Ego translations distribution", size=14)
plt.show()

#### Ego rotations

Let's also plot Ego rotations components distributions. The rotation matrix is 3 x 3.

In [ ]:
fig, ax = plt.subplots(3,3,figsize=(16,16))
colors = ['red', 'blue', 'green', 'magenta', 'orange', 'darkblue', 'black', 'cyan', 'darkgreen']
for i in range(0,3):
    for j in range(0,3):
        df = frames_df['ego_rotation'].apply(lambda x: x[i][j])
        plt.subplot(3,3,i * 3 + j + 1)
        sns.distplot(df, hist=False, color = colors[ i * 3 + j  ])
        plt.xlabel(f'r[ {i + 1} ][ {j + 1} ]')
fig.suptitle("Ego rotation angles distribution", size=14)
plt.show()

#### Trafic lights faces index interval

In [ ]:
frames_df['tlfii0'] = frames_df['traffic_light_faces_index_interval'].apply(lambda x: x[0])
frames_df['tlfii1'] = frames_df['traffic_light_faces_index_interval'].apply(lambda x: x[1])
sns.set_style('whitegrid')
plt.figure()
fig, ax = plt.subplots(1,1,figsize=(8,8))
plt.scatter(frames_df['tlfii0'], frames_df['tlfii1'], marker='+')
plt.xlabel('Trafic lights faces index interval [0]', fontsize=11); plt.ylabel('Trafic lights faces index interval [1]', fontsize=11)
plt.title("Trafic lights faces index interval")
plt.show()

#### Agents index interval

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
colors = ['cyan', 'darkgreen']
for i in range(0,2):
    df = frames_df['agent_index_interval'].apply(lambda x: x[i])
    plt.subplot(1, 2, i + 1)
    sns.distplot(df, hist=False, color = colors[ i ])
    plt.xlabel(f'agent index interval [ {i} ]')
fig.suptitle("Agent index interval", size=14)
plt.show()

### 4.2. Visualize animated scene


We define two utility functions:   
* one function to display animated scenes, using animation from matplotlib;  
* one function to assemble the animation; this function receives as parameters the scene index and the map type (either `semantic` or `satellite`).  

In [ ]:
def show_scene_animated(images):

    def animate(i):
        im.set_data(images[i])
 
    fig, ax = plt.subplots()
    im = ax.imshow(images[0])
    
    return animation.FuncAnimation(fig, animate, frames=len(images), interval=60)

def prepare_scene_for_animation(scene_index=20,map_type="py_semantic",show_trajectory=False):
    cfg["raster_params"]["map_type"] = map_type
    rast = build_rasterizer(cfg, dm)
    dataset = EgoDataset(cfg, chunked_dataset, rast)
    scene_idx = scene_index
    indexes = dataset.get_scene_indices(scene_idx)
    images = []

    for idx in indexes:

        data = dataset[idx]
        im = data["image"].transpose(1, 2, 0)
        im = dataset.rasterizer.to_rgb(im)
        target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
        center_in_pixels = np.asarray(cfg["raster_params"]["ego_center"]) * cfg["raster_params"]["raster_size"]
        if show_trajectory:
            draw_trajectory(on_image=im, positions=target_positions_pixels, rgb_color=TARGET_POINTS_COLOR, radius=1, yaws=data["target_yaws"])
        clear_output(wait=True)
        images.append(PIL.Image.fromarray(im[::-1]))    
    return images

Let's visualize two different scenes, using semantic map type.

In [ ]:
semantic_images_animation = show_scene_animated(prepare_scene_for_animation(10,"py_semantic"))

In [ ]:
HTML(semantic_images_animation.to_jshtml())

In [ ]:
semantic_images_animation = show_scene_animated(prepare_scene_for_animation(20,"py_semantic"))

In [ ]:
HTML(semantic_images_animation.to_jshtml())

In [ ]:
semantic_images_animation = show_scene_animated(prepare_scene_for_animation(30,"py_semantic"))

In [ ]:
HTML(semantic_images_animation.to_jshtml())

In [ ]:
semantic_images_animation = show_scene_animated(prepare_scene_for_animation(40,"py_semantic"))

In [ ]:
HTML(semantic_images_animation.to_jshtml())

Let's look now to the same scenes, but using satellite map type.

In [ ]:
satellite_images_animation = show_scene_animated(prepare_scene_for_animation(10, "py_satellite"))

In [ ]:
HTML(satellite_images_animation.to_jshtml())

In [ ]:
satellite_images_animation = show_scene_animated(prepare_scene_for_animation(20, "py_satellite"))

In [ ]:
HTML(satellite_images_animation.to_jshtml())

In [ ]:
satellite_images_animation = show_scene_animated(prepare_scene_for_animation(30, "py_satellite"))

In [ ]:
HTML(satellite_images_animation.to_jshtml())

In [ ]:
satellite_images_animation = show_scene_animated(prepare_scene_for_animation(40, "py_satellite"))

In [ ]:
HTML(satellite_images_animation.to_jshtml())

## 5. References

[1] Lyft Understanding the data and EDA, https://www.kaggle.com/nxrprime/lyft-understanding-the-data-and-eda  
[2] Lyft Scenes Visualizations, https://www.kaggle.com/jpbremer/lyft-scene-visualisations/  
[3] Lyft l5kit, https://github.com/lyft/l5kit  
[4] Lyft l5kit data visualization, https://github.com/lyft/l5kit/blob/master/examples/visualisation/visualise_data.ipynb  
[5] Lyft l5kit agents motion prediction, https://github.com/lyft/l5kit/blob/master/examples/agent_motion_prediction/agent_motion_prediction.ipynb   